In [ ]:
import numpy as np
import cv2
from skimage import exposure
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter
import matplotlib.pyplot as plt
import spectral as sp
import os
from skimage.measure import label, regionprops
import pandas as pd

In [ ]:
# Step 1: Load hyperspectral data
def load_hyperspectral_data(bil_path):
    
    hsi = sp.open_image(bil_path)
    hsi_data = hsi.load()
    
    return hsi_data


# Step 2: Flat-field correction
def flat_field_correction(raw_cube, dark_ref, white_ref):
    
    corrected_cube = (raw_cube - dark_ref) / (white_ref - dark_ref)
    
    return corrected_cube


# Step 3: Calculate Normalized Band Difference (NBD)
def calculate_nbd(reflectance_cube, band1_idx, band2_idx):
    
    r_band1 = reflectance_cube[:, :, band1_idx]
    r_band2 = reflectance_cube[:, :, band2_idx]
    
    # Avoid division by zero
    epsilon = 1e-10
    denominator = r_band2 + r_band1 + epsilon
    
    nbd_image = (r_band2 - r_band1) / denominator
    return nbd_image


# Step 4: Enhance Image Contrast
def enhance_contrast(image):
    
    return exposure.equalize_hist(image)


# Step 5: Segment using thresholding
def segment_image(image):
    
    _, binary_mask = cv2.threshold((image * 255).astype('uint8'), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return binary_mask


# Step 6: Morphological operations
def clean_segmentation(binary_mask):
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    
    cleaned_mask = cv2.erode(binary_mask, kernel, iterations=2)
    cleaned_mask = cv2.dilate(cleaned_mask, kernel, iterations=2)
    
    return cleaned_mask



def detect_circles(cleaned_mask, radius_range):
    
    # Compute Hough Circle Transform
    hough_radii = np.arange(radius_range[0], radius_range[1], 1)
    hough_res = hough_circle(cleaned_mask, hough_radii)

    # Identify the most prominent circles
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=600)#change if some bb not detected

    # Create a mask to keep only the circular regions
    circle_mask = np.zeros_like(cleaned_mask, dtype=bool)
    
    for center_y, center_x, radius in zip(cy, cx, radii):
        
        rr, cc = circle_perimeter(center_y, center_x, radius, shape=circle_mask.shape)
        circle_mask[rr, cc] = True

    # Fill the circles to create a binary mask
    filled_circle_mask = np.zeros_like(cleaned_mask, dtype=np.uint8)  # Convert to uint8
    
    for center_y, center_x, radius in zip(cy, cx, radii):
        
        rr, cc = circle_perimeter(center_y, center_x, radius, shape=filled_circle_mask.shape)
        filled_circle_mask[rr, cc] = 255  # Set perimeter pixels to 255
        
        cv2.circle(filled_circle_mask, (center_x, center_y), radius, 255, thickness=-1)  # Fill circle with 255

    return filled_circle_mask



def calculate_mean_spectra_from_segments(filled_circle_mask, reflectance_cube):
    
    # Label connected components in the circular mask
    labeled_mask = label(filled_circle_mask)
    regions = regionprops(labeled_mask)

    mean_spectra = []

    for region in regions:
        
        # Get the mask for this specific region
        individual_mask = labeled_mask == region.label

        # Extract the reflectance values for pixels within the region
        blueberry_pixels = reflectance_cube[individual_mask]

        # Calculate the mean spectrum for this region
        mean_spectrum = np.mean(blueberry_pixels, axis=0)

        # Append to the list
        mean_spectra.append(mean_spectrum)

    return mean_spectra

In [ ]:
def process_hyperspectral_cube(file_path, white_ref_path, dark_ref, band1_idx, band2_idx, radius_range):
    
    # Load raw data and white reference
    raw_cube = load_hyperspectral_data(file_path)
    white_ref = load_hyperspectral_data(white_ref_path)

    # Visualize raw cube
    plt.imshow(raw_cube[:, :, band1_idx], cmap='gray')
    plt.title("Raw Cube (Band {})".format(band1_idx))
    plt.show()

    # Flat-field correction
    reflectance_cube = flat_field_correction(raw_cube, dark_ref, white_ref)
    plt.imshow(reflectance_cube[:, :, band1_idx], cmap='gray')
    plt.title("Flat-Field Corrected Cube (Band {})".format(band1_idx))
    plt.show()

    # Calculate NBD
    nbd_image = calculate_nbd(reflectance_cube, band1_idx, band2_idx)
    plt.imshow(nbd_image, cmap='gray')
    plt.title("Normalized Band Difference (NBD)")
    plt.colorbar()
    plt.show()

    # Enhance contrast
    enhanced_image = enhance_contrast(nbd_image)
    plt.imshow(enhanced_image, cmap='gray')
    plt.title("Enhanced Contrast Image")
    plt.colorbar()
    plt.show()

    # Segment
    segmented_mask = segment_image(enhanced_image)
    plt.imshow(segmented_mask, cmap='gray')
    plt.title("Segmented Binary Mask")
    plt.show()

    # Clean segmentation
    cleaned_mask = clean_segmentation(segmented_mask)
    plt.imshow(cleaned_mask, cmap='gray')
    plt.title("Cleaned Segmentation Mask")
    plt.show()

    # Detect circles
    final_mask = detect_circles(cleaned_mask, radius_range)
    plt.imshow(final_mask, cmap='gray')
    plt.title("Detected Circles")
    plt.show()

    # Calculate mean spectra directly
    mean_spectra = calculate_mean_spectra_from_segments(final_mask, reflectance_cube)

    # Add a label column
    label = "bad_blueberry"
    labeled_spectra = [np.append(spectrum, label) for spectrum in mean_spectra]

    # Convert to DataFrame for analysis
    spectra_df = pd.DataFrame(labeled_spectra)

    # Save as CSV (no header row)
    output_csv_path = "SM/bad_blueberry_sm_169_210.csv"
    spectra_df.to_csv(output_csv_path, index=False, header=False)
    
    print(f"Spectral features saved to {output_csv_path}")

    # Save as .npy file
    output_npy_path = "SM/bad_blueberry_sm_169_210.npy"
    
    np.save(output_npy_path, np.array(labeled_spectra))
    print(f"Spectral features saved to {output_npy_path}")

    return final_mask, spectra_df

In [ ]:
# Dataset
dataset_path = '<path_to_file>' #must change according to dataset 

bil_file_path = os.path.join(dataset_path, "Bad Blueberries 169-210.bil.hdr") #must change according to dataset 
#white_ref_path = os.path.join(dataset_path, "WhiteReference.bil.hdr")
white_ref_path = os.path.join(dataset_path, "WhiteReference_1600.bil.hdr") #must change according to dataset 

near_zero = 1e-6
#dark_ref = np.full((1800, 1600, 462), near_zero) #must change according to dataset 
dark_ref = np.full((1600, 1600, 462), near_zero) #must change according to dataset 

# Process the cube
band1_idx = 216   # 680nm
band2_idx = 380   # 900nm
radius_range = (40, 85)

final_mask, spectra_df = process_hyperspectral_cube(bil_file_path, white_ref_path, dark_ref, band1_idx, band2_idx, radius_range)

In [ ]:
data = np.load('SM/bad_blueberry_sm_127_168.npy', allow_pickle= True)
data.shape

In [ ]:
def plot_random_spectral_curves(data, num_samples):
    
    # Exclude the label column
    spectral_data = data[:, :-1]
    labels = data[:, -1]

    # Randomly choose `num_samples` rows
    random_indices = np.random.choice(spectral_data.shape[0], num_samples, replace=False)

    plt.figure(figsize=(10, 5))

    for i, idx in enumerate(random_indices):
        
        # Extract the label
        label = labels[idx]

        # Extract the spectral values
        spectral_values = spectral_data[idx].astype(float)
        wavelengths = np.arange(1, spectral_values.shape[0] + 1)

        # Plot the spectral curve
        plt.plot(wavelengths, spectral_values, label=f"Sample {idx} - Label: {label}")

    plt.title(f"Spectral Curves for {num_samples} Random Samples")
    plt.xlabel("Bands")
    plt.ylabel("Mean Reflectance")
    plt.grid(True)
    plt.legend()
    plt.show()
    
plot_random_spectral_curves(data, num_samples = 10)